In [ ]:
from google.colab import drive
import shutil, zipfile, os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

# 1) Mount & unpack
drive.mount('/content/drive')
shutil.copy("/content/drive/MyDrive/DATASET.zip", "/content/")
with zipfile.ZipFile("DATASET.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")

Mounted at /content/drive


In [ ]:
def load_images_from_folder(folder_path, image_size=(224, 224)):
    X, y = [], []
    class_names = sorted(os.listdir(folder_path))
    class_map = {n:i for i,n in enumerate(class_names)}
    for cls in class_names:
        folder = os.path.join(folder_path, cls)
        for f in os.listdir(folder):
            try:
                img = Image.open(os.path.join(folder, f)).convert('RGB').resize(image_size)
                X.append(np.array(img))
                y.append(class_map[cls])
            except:
                continue
    return np.array(X)/255.0, np.array(y), class_map

X, y, class_map = load_images_from_folder("dataset/DATASET")
class_names = list(class_map.keys())

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=2/3, stratify=y_temp, random_state=42)


In [ ]:
class_weights = compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)


In [ ]:
input_shape = (224, 224, 3)
num_classes = len(class_map)

# Load the pretrained MobileNetV2, omit its top
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False  # freeze backbone

# Add custom head
inputs = Input(shape=input_shape)
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)

x = layers.Dense(1024, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs, name='MobileNetV2_Fashion')

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "MobileNetV2_Fashion"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     1,311,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,105,802 (15.66 MB)

 Trainable params: 1,844,746 (7.04 MB)

 Non-trainable params: 2,261,056 (8.63 MB)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=50,
    class_weight=class_weights,
    verbose=1
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.1304 - loss: 3.5895 - val_accuracy: 0.4600 - val_loss: 1.7261
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 405ms/step - accuracy: 0.4435 - loss: 1.7892 - val_accuracy: 0.7000 - val_loss: 1.2082
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 7s 330ms/step - accuracy: 0.6244 - loss: 1.3128 - val_accuracy: 0.7900 - val_loss: 0.8988
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 399ms/step - accuracy: 0.6738 - loss: 1.0294 - val_accuracy: 0.8300 - val_loss: 0.6974
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 376ms/step - accuracy: 0.7357 - loss: 0.8248 - val_accuracy: 0.8500 - val_loss: 0.5660
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 327ms/step - accuracy: 0.8021 - loss: 0.6664 - val_accuracy: 0.8600 - val_loss: 0.4907
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 403ms/step - accuracy: 0.7906 - loss: 0.7048 - val_accuracy: 0.8800 - val_loss: 0.4280
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 421ms/step - accuracy: 0.7960 - loss: 0.5902 - val_accuracy: 0.87

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 617ms/step - accuracy: 0.8950 - loss: 0.3431
Test Accuracy: 0.8900


In [ ]:
model.save('/content/drive/My Drive/Collab Notebooks/fashion_product_mobilenet.h5')


In [ ]:
!pip install gradio
import gradio as gr

def predict_fashion(img):
    img = img.resize((224,224))
    arr = np.array(img)/255.0
    pred = model.predict(arr[np.newaxis,...])
    return class_names[np.argmax(pred)]

iface = gr.Interface(
    fn=predict_fashion,
    inputs=gr.Image(type="pil"),
    outputs="text",
    live=True,
    title="Fashion Classifier (MobileNetV2)",
    description="Upload an image to classify."
)
iface.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 132.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.3 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eaed9d75fb75856eac.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from sklearn.metrics import (
    precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)
import numpy as np

# Predict probabilities
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)
test_preds = model.predict(X_test)

# Convert probabilities to predicted class labels
train_preds_class = np.argmax(train_preds, axis=1)
val_preds_class = np.argmax(val_preds, axis=1)
test_preds_class = np.argmax(test_preds, axis=1)

# True labels (assuming already integers)
y_train_class = y_train
y_val_class = y_val
y_test_class = y_test

# AUC Scores
train_auc = roc_auc_score(y_train, train_preds, multi_class='ovr')
val_auc = roc_auc_score(y_val, val_preds, multi_class='ovr')
test_auc = roc_auc_score(y_test, test_preds, multi_class='ovr')

# Accuracy
train_accuracy = np.mean(train_preds_class == y_train_class)
val_accuracy = np.mean(val_preds_class == y_val_class)
test_accuracy = np.mean(test_preds_class == y_test_class)

# Loss
train_loss = model.evaluate(X_train, y_train, verbose=0)[0]
val_loss = model.evaluate(X_val, y_val, verbose=0)[0]
test_loss = model.evaluate(X_test, y_test, verbose=0)[0]

# Precision, Recall, F1-Score (Weighted & Macro)
precision_weighted = precision_score(y_test_class, test_preds_class, average='weighted')
recall_weighted = recall_score(y_test_class, test_preds_class, average='weighted')
f1_weighted = f1_score(y_test_class, test_preds_class, average='weighted')

precision_macro = precision_score(y_test_class, test_preds_class, average='macro')
recall_macro = recall_score(y_test_class, test_preds_class, average='macro')
f1_macro = f1_score(y_test_class, test_preds_class, average='macro')

# Sensitivity = Recall (Weighted)
sensitivity = recall_weighted

# Specificity (approximate for multiclass): TN / (TN + FP) averaged
conf_matrix = confusion_matrix(y_test_class, test_preds_class)
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)
specificity = np.mean(TN / (TN + FP))

# Classification Miss Rate
miss_rate = 1 - sensitivity

# Output all metrics
print(f"🔹 AUC - Train: {train_auc:.4f}, Validation: {val_auc:.4f}, Test: {test_auc:.4f}")
print(f"🔹 Accuracy - Train: {train_accuracy:.4f}, Validation: {val_accuracy:.4f}, Test: {test_accuracy:.4f}")
print(f"🔹 Loss - Train: {train_loss:.4f}, Validation: {val_loss:.4f}, Test: {test_loss:.4f}")
print(f"🔹 Precision (Weighted): {precision_weighted:.4f}, (Macro): {precision_macro:.4f}")
print(f"🔹 Recall (Sensitivity) (Weighted): {recall_weighted:.4f}, (Macro): {recall_macro:.4f}")
print(f"🔹 F1-Score (Weighted): {f1_weighted:.4f}, (Macro): {f1_macro:.4f}")
print(f"🔹 Specificity (approx): {specificity:.4f}")
print(f"🔹 Classification Miss Rate: {miss_rate:.4f}")
print("\n🔹 Confusion Matrix:\n", conf_matrix)
print("\n🔹 Classification Report:\n", classification_report(y_test_class, test_preds_class))


22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 212ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 575ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 304ms/step
🔹 AUC - Train: 0.9998, Validation: 0.9959, Test: 0.9899
🔹 Accuracy - Train: 0.9871, Validation: 0.8900, Test: 0.8900
🔹 Loss - Train: 0.0315, Validation: 0.2384, Test: 0.3761
🔹 Precision (Weighted): 0.8974, (Macro): 0.8766
🔹 Recall (Sensitivity) (Weighted): 0.8900, (Macro): 0.8750
🔹 F1-Score (Weighted): 0.8907, (Macro): 0.8726
🔹 Specificity (approx): 0.9879
🔹 Classification Miss Rate: 0.1100

🔹 Confusion Matrix:
 [[20  0  0  0  0  0  0  0  0  0]
 [ 0 13  0  0  0  0  0  7  0  0]
 [ 0  0 30  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  3]
 [ 0  0  0  0 20  0  0  0  0  0]
 [ 1  0  0  1  0 17  0  0  0  1]
 [ 0  0  0  0  2  0 18  0  0  0]
 [ 0  3  0  0  0  0  0 17  0  0]
 [ 0  0  0  0  0  0  0  0 20  0]
 [ 0  0  0  4  0  0  0  0  0 16]]

🔹 Classification Report:
               precision    recall  f1-score   support

           0       0.95      1.00      0.98        2

In [ ]:
from sklearn.metrics import (
    confusion_matrix, fbeta_score, precision_score, recall_score, f1_score
)
import numpy as np

# Predict classes
test_preds = model.predict(X_test)
test_preds_class = np.argmax(test_preds, axis=1)
y_true = y_test  # assumed to be integer-encoded

# Confusion Matrix
cm = confusion_matrix(y_true, test_preds_class)
TP = np.diag(cm)
FP = cm.sum(axis=0) - TP
FN = cm.sum(axis=1) - TP
TN = cm.sum() - (FP + FN + TP)

# Small value to prevent divide-by-zero
eps = 1e-10

# F2-Score (beta=2)
f2_score_weighted = fbeta_score(y_true, test_preds_class, average='weighted', beta=2)

# Fβ-Score for general beta (set your desired beta)
beta = 0.5
f_beta_score_weighted = fbeta_score(y_true, test_preds_class, average='weighted', beta=beta)

# Positive Predictive Value (Precision)
ppv = precision_score(y_true, test_preds_class, average='weighted')

# Negative Predictive Value (NPV)
npv = np.mean(TN / (TN + FN + eps))

# False Positive Rate (FPR)
fpr = np.mean(FP / (FP + TN + eps))

# False Negative Rate (FNR)
fnr = np.mean(FN / (FN + TP + eps))

# Likelihood Ratios
sensitivity = recall_score(y_true, test_preds_class, average='weighted')
specificity = np.mean(TN / (TN + FP + eps))

lr_positive = sensitivity / (1 - specificity + eps)
lr_negative = (1 - sensitivity) / (specificity + eps)

# Fowlkes–Mallows Index (FMI)
precision_macro = precision_score(y_true, test_preds_class, average='macro')
recall_macro = recall_score(y_true, test_preds_class, average='macro')
fmi = np.sqrt(precision_macro * recall_macro)

# Print Results
print(f"🔹 F2-Score (Weighted): {f2_score_weighted:.4f}")
print(f"🔹 F{beta}-Score (Weighted): {f_beta_score_weighted:.4f}")
print(f"🔹 Positive Predictive Value (PPV): {ppv:.4f}")
print(f"🔹 Negative Predictive Value (NPV): {npv:.4f}")
print(f"🔹 False Positive Rate (FPR): {fpr:.4f}")
print(f"🔹 False Negative Rate (FNR): {fnr:.4f}")
print(f"🔹 Likelihood Ratio Positive (LR+): {lr_positive:.4f}")
print(f"🔹 Likelihood Ratio Negative (LR−): {lr_negative:.4f}")
print(f"🔹 Fowlkes–Mallows Index (FMI): {fmi:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
🔹 F2-Score (Weighted): 0.8896
🔹 F0.5-Score (Weighted): 0.8940
🔹 Positive Predictive Value (PPV): 0.8974
🔹 Negative Predictive Value (NPV): 0.9879
🔹 False Positive Rate (FPR): 0.0121
🔹 False Negative Rate (FNR): 0.1250
🔹 Likelihood Ratio Positive (LR+): 73.6998
🔹 Likelihood Ratio Negative (LR−): 0.1113
🔹 Fowlkes–Mallows Index (FMI): 0.8758
